In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import re
import itertools
import time

In [2]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [4]:
embed = hub.Module(module_url)

INFO:tensorflow:Using C:\Users\zlai\AppData\Local\Temp\tfhub_modules to cache modules.


In [5]:
os.getcwd()

'C:\\Users\\zlai\\Documents\\repo\\ML-RN-Project'

In [11]:
os.chdir('/Users/zlai/Documents/repo/ML-RN-Project/tasks_1-20_v1-2/en-10k')

In [12]:
os.getcwd()

'C:\\Users\\zlai\\Documents\\repo\\ML-RN-Project\\tasks_1-20_v1-2\\en-10k'

In [13]:
f = open('qa2_two-supporting-facts_train.txt','r')

In [14]:
babi = f.readlines()

In [15]:
def _split_paragraphs(path_to_file):
        """
        split into paragraphs as babi dataset consists of multiple 1~n sentences

        Args
            file_path: path of the data

        Returns
            paragraphs: list of paragraph
        """
        with open(path_to_file, 'r') as f:
            babi = f.readlines()
        paragraph = []
        paragraphs = []
        alphabet = re.compile('[a-zA-Z]')
        for d in babi:
            if d.startswith('1 '):
                if paragraph:
                    paragraphs.append(paragraph)
                paragraph = []
            mark = re.search(alphabet, d).span()[0]
            paragraph.append(d[mark:])
        return paragraphs
    
def _split_clqa(paragraphs, show_print= True):
        """
        for each paragraph, split into context, label, question and answer

        Args
            paragraphs: list of paragraphs

        Returns
            context: list of contexts
            label: list of labels
            question: list of questions
            answer: list of answers
        """
        context = []
        label = []
        question = []
        answer = []
        for paragraph in paragraphs:
            for i, sent in enumerate(paragraph):
                if '?' in sent:
                    related_para = [para.strip().lower() for para in paragraph[:i] if '?' not in para][::-1]
                    if len(related_para) > 20:
                        related_para = related_para[:20]
                    context.append(related_para)
                    label.append([i for i in range(len(related_para))])
                    q_a_ah = sent.split('\t')
                    question.append(q_a_ah[0].strip().lower())
                    answer.append(q_a_ah[1].strip().lower())
        # check
        if show_print:
            if (len(question) == len(answer)) & (len(answer) == len(context)) & (len(context) == len(label)):
                print("bAbI is well separated into question, answer, context, and label!")
                print("total: {}".format(len(label)))
            else:
                print("Something is missing! check again")
                print("the number of questions: {}".format(len(question)))
                print("the number of answers: {}".format(len(answer)))
                print("the number of contexts: {}".format(len(context)))
                print("the number of labels: {}".format(len(label)))
        return context, label, question, answer    

In [16]:
p = _split_paragraphs('qa2_two-supporting-facts_train.txt')

In [18]:
c, l, q, a = _split_clqa(p)
os.chdir('/Users/zlai/Documents/repo/ML-RN-Project')
np.save('c', c)
np.save('l', l)
np.save('q', q)
np.save('a', a)

bAbI is well separated into question, answer, context, and label!
total: 9995


In [13]:
len(c)

9995

In [14]:
c1 = c[0:100]
l1 = l[0:100]
q1 = q[0:100]
a1 = a[0:100]
os.chdir('/home/nvaitc/Documents/repos/Relation-Network/')
np.save('a1', a1)

In [15]:
def context_embedding(c):
    """
    Converts context to embeddings using USE.
    Sentences are obtained from the output of 
    the _split_clqa function. Each sentence 
    embedding has dim 512.
    Input(s):
    - c : context
    """
    embedded_c = []
    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)

    # Convert paragraphs to embedding using USE
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        for para in c:
            embedding = session.run(embed(para))
            embedding = embedding.tolist()
            embedded_c.append(embedding)
    return embedded_c       

In [106]:
def context_embedding_fast(c):
    """
    Converts context to embeddings using USE.
    Sentences are obtained from the output of 
    the _split_clqa function. Each sentence 
    embedding has dim 512.
    Input(s):
    - c : context
    """
    embedded_c = []
    t1 = time.time()
    for i in c:
        with tf.Graph().as_default():
            embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
            output = embed(i)

            with tf.Session() as session:
                session.run([tf.global_variables_initializer(), tf.tables_initializer()])
#                 t1 = time.time()
                message_embeddings = session.run(output)
                message_embedding = message_embedding.tolist()
                embedded_c.append(message_embeddings)
#                 print(time.time() - t1)
    print(time.time() - t1)
    return embedded_c      

In [107]:
def add_labels(embedded_c):
    """
    Adds the labels in the front of the context embeddings.
    Output of embedded_c here follows the ordering from 
    earliest to latest.
    Input(s):
    - embedded_c : embeddings from USE
    """
    embedded_c_list = []
    for j, para in enumerate(embedded_c):
        a = np.array(para).tolist()
        for i, sent in enumerate(a):
            print(type(a[-(i+1)]))
            a[-(i+1)].insert(0, i)
            embedded_c[j][i] = a[-(i+1)]
    return embedded_c        

In [18]:
def question_embedding(q):
    """
    Converts questions to embeddings using USE.
    Sentences are obtained from the output of 
    the _split_clqa function. Each sentence 
    embedding has dim 512.
    Input(s):
    - q : question
    """
    embedded_q = []
    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)

    # Convert paragraphs to embedding using USE
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        for para in q:
            para = [para]
            embedding = session.run(embed(para))
            embedding = embedding.tolist()
            embedded_q.append(embedding)
    return embedded_q       

In [105]:
def question_embedding_fast(q):
    """
    Converts questions to embeddings using USE.
    Sentences are obtained from the output of 
    the _split_clqa function. Each sentence 
    embedding has dim 512.
    Input(s):
    - q : question
    """
    embedded_q = []
    t1 = time.time()
    for i in q:
        with tf.Graph().as_default():
            embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
            output = embed([i])

            with tf.Session() as session:
                session.run([tf.global_variables_initializer(), tf.tables_initializer()])
#                 t1 = time.time()
                message_embeddings = session.run(output)
                message_embedding = message_embedding.tolist()
                embedded_q.append(message_embeddings)
#                 print(time.time() - t1)
    print(time.time() - t1)
    return embedded_q

In [124]:
def convert_to_RN_input(embedded_c, embedded_q, answer):
    """
    Returns the input for RN which has dim (n, 190, 1536), 
    where n is the number of context and 190=20C2.
    Input(s):
    - embedded_c: list of context (up to 20 sentences) 
                  for each different question.  
    - embedded_q: embedding for the related question to 
                  the given embedded_c.
    - answer: corresponding answer to the embedded c and q.              
    """
    object_pairs = []

    for i in embedded_c:
        object_pair = list(itertools.combinations(i, 2))
        object_pairs.append(object_pair)
    
    rn_inputs = []
    m = len(object_pairs)
    z = np.zeros((m, 190, 1536))    
    
    for i, object_pair in enumerate(object_pairs):
        n = len(object_pair)
        for j in range(n):
            l = np.array(object_pair[j][0] + object_pair[j][1] + embedded_q[i][0])
#             z[i][j] = np.array(object_pair[j][0] + object_pair[j][1] + embedded_q[i][0])
            z[i][j] = np.concatenate((object_pair[j][0], object_pair[j][1], embedded_q[i][0]))
    return z

In [22]:
embedded_c = context_embedding_fast(c)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver

...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver

In [24]:
os.chdir('/home/nvaitc/Documents/repos/Relation-Network')
np.save('embedded_c', embedded_c)

In [111]:
embedded_c = np.load('embedded_c.npy')

In [112]:
embedded_c[0].shape

(6, 512)

In [108]:
tag_embedded_c = add_labels(embedded_c.tolist())

<class 'list'>


ValueError: cannot copy sequence with size 513 to array axis with dimension 512

In [29]:
embedded_q = question_embedding_fast(q)

53414.87280678749


In [30]:
os.chdir('/home/nvaitc/Documents/repos/Relation-Network')
np.save('embedded_q', embedded_q)

In [125]:
RN_input = convert_to_RN_input(embedded_c, embedded_q, a)

In [126]:
os.chdir('/home/nvaitc/Documents/repos/Relation-Network')
np.save('RN_input', RN_input)

In [127]:
RN_input.shape

(9995, 190, 1536)

In [ ]:
# To import out the preprocessed data into csv format
# uncomment to use
# import csv

# with open("output.csv", "w") as f:
#     writer = csv.writer(f)
#     writer.writerows(out)